# Tratamento de dados - Secretaria de segurança do estado do Pará

## Descrição do notebook

O objetivo desse notebook é realizar o tratamento dos dados fornecidos pela Secretaria de Segurança Pública e Defesa Social do Estado do Pará para que possa ser analisado juntamente com os dados do estado paranaense.

Nessa versão, o arquivo utilizado é um XLSX e para manipulação de dados é utilizado o pacote **pandas**. 

**Observação:** O código contido nesse notebook foi baseado na solução desenvolvida por Mariana Kniss (https://www.linkedin.com/in/mariana-kniss-471ba0196/), para validar o conteúdo de seu trabalho de conclusão de curso, intitulado "ANÁLISE DE DADOS CRIMINAIS SOBRE VIOLÊNCIA CONTRA A MULHER NOS ESTADOS PARÁ E PARANÁ".

## Sumário

1. [Importação dos módulos e pacotes necessários](#importacao-dos-modulos-e-pacotes-necessarios)
2. [Carregamento dos dados](#carregamento-dos-dados)
4. [Visão geral do conjunto de dados](#visao-geral-do-conjunto-de-dados)
5. [Filtros, tratamentos e padronização de valores](#filtros-tratamentos-e-padronizacao-de-valores)
6. [Ordenação das colunas](#ordernacao-das-colunas)
7. [Exportação dos dados formatados](#exportacao-dos-dados-formatados)

----


## Importação dos módulos e pacotes necessários <a id="importacao-dos-modulos-e-pacotes-necessarios"></a>

In [ ]:
# Pandas (https://pandas.pydata.org/)
import pandas as pd

# Datetime (https://docs.python.org/3/library/datetime.html)
import datetime as dt

## Carregamento dos dados

Inicialmente, realizada com **spark** e utilizando um arquivo do tipo **CSV**, o carregamento foi alterado para adotar o **pandas**, utilizando um arquivo do tipo **Excel (XLSX)**.

In [ ]:
# Carregamento dos dados utilizando o arquivo XLSX e pandas
dados = pd.read_csv('./dados/PA/raw_BD_VD_2010a2022_PA.csv', sep='@')

In [ ]:
# Exibe a quantidade de registros encontrados
len(dados)

In [ ]:
# Exibe o tipo de variável
type(dados)

**Observação:** o método read_csv() do pandas carregou corretamente o número de registros: 196250.

----

## Visão geral do conjunto de dados <a id="visao-geral-do-conjunto-de-dados"></a>

In [ ]:
# Contagem do número de variáveis
v = len(dados.columns)
# Contagem do número de registros
n = len(dados)

print(f'O conjunto de dados possui {n} registros e {v} variáveis.')

In [ ]:
# Visualização do dataframe
dados.head(10)

In [ ]:
# Descrição dos tipos dos dados
dados.info()

## Filtros, tratamentos e padronização de valores <a id="filtros-tratamentos-e-padronizacao-de-valores"></a>

### Filtros

#### Sexo das vítimas

In [ ]:
# Contagem dos valores distintos na coluna 'vit_sexo'
dados.vit_sexo.value_counts()

Filtrando apenas registros nos quais as vítimas são do sexo feminino.

In [ ]:
# Filtrando a base para que "vit_sexo" = "F"
dados_filtro_vit_F = dados[dados['vit_sexo']  == 'F']

# Exibindo a quantidade de registros restantes
n_feminino = len(dados_filtro_vit_F)

print(f'Foram retirados {n - n_feminino}, restando {n_feminino} registros válidos.')

#### Sexo dos autores

In [ ]:
# Contagem dos valores distintos na coluna 'aut_sexo'
dados_filtro_vit_F.aut_sexo.value_counts()

Selecionando apenas regitros onde o sexo dos autores é diferente de feminino.

In [ ]:
# Filtrando a base para que "aut_sexo" diferente "F"
dados_filtro_aut_nF = dados_filtro_vit_F.loc[(dados_filtro_vit_F['aut_sexo'] != 'F') | (dados_filtro_vit_F['aut_sexo'].isnull())]

# Exibindo o dataset VCM
n_autor_nf = len(dados_filtro_aut_nF)

print(f'Foram retirados {n_feminino - n_autor_nf}, restando {n_autor_nf} registros válidos.')

**Comentários:** A partir desse procedimento, os valores obtidos para a utilização do pandas são discrepantes se comparados ao uso do pyspark. 

Mesmo assim, ao considerar que a quantidade de registros com o valor 'F' para a coluna 'aut_sexo' é 3272, ao subtrair esse valor dos 171066 registros válidos até esse ponto, o novo total seria 167794. Esse valor foi obtido pelo procedimento realizado aqui.

#### Ano do fato

A base de dados contava com registros de 2010 a 2023, porém foram selecionados apenas os casos ocorridos entre 2018 e 2022.

In [ ]:
# Filtrando a base para que ano registro >= 2018
dados_filtro_ano_registro = dados_filtro_aut_nF[(dados_filtro_aut_nF['ano_fato'] >= 2018) & (dados_filtro_aut_nF['ano_fato'] <= 2022)]

# Exibindo o dataset VCM
n_registros_apos_2018 = len(dados_filtro_ano_registro)

print(f'Foram retirados {n_autor_nf - n_registros_apos_2018}, restando {n_registros_apos_2018} registros válidos.')

#### Especificação do fato

Para selecionar apenas os registros de violência contra mulher (VCM), foram mantidas apenas as linhas que contivesse uma das expressões abaixo na coluna 'especificacao_fato':

- "mulher"
- "penha"
- "feminicídio"
- "femicídio"
- "feminicidio"
- "femicidio"

In [ ]:
# Selecionando a base de VCM com base na variavel 'especializacao_fato', utilizando como filtro as palavras chave
palavras_chave = ["mulher", "feminicídio", "penha", "femicidio", "feminicidio", "femicídio"]

raw_VCM = dados_filtro_ano_registro[(dados_filtro_ano_registro['especializacao_fato'].notnull()) & (dados_filtro_ano_registro['especializacao_fato'].str.lower().str.contains('|'.join(palavras_chave)))]

In [ ]:
n_vcm = len(raw_VCM)
print(f'Ao final, foram mantidos {n_vcm} registros. Ou seja, foram removidos {n_registros_apos_2018 - n_vcm} registros que não se encaixaram nos critérios definidos.')

In [ ]:
# Contagem dos valores distintos na coluna 'especializacao_fato'
raw_VCM['especializacao_fato'].value_counts()

### Tratamentos

#### Removação de atributos não relevantes para as análises

In [ ]:
# Remoção de colunas desnecessárias
colunas_para_remover = ['data_inst_proc', # Data de instauração do processo investigativo
						'data_concl_proc', # Data de conclusão do processo investigativo
						'sit_proc', # Situação do processo investigativo
						'grupo_ocorrencia', # Agrupamento (categorização) das ocorrências
						'sub_grupo', # Sub-agrupamento (categorização) das ocorrências
						'reg_integracao', # Regional de integração
						'risp', # RISP
						'aisp', # AISP
						'rua_fato', # Logradouro no qual aconteceu o fato
						'empresa', # Empresa de transporte relacionada ao fato
						'linha', # Linha de transporte relacionada ao fato
						'tipo_transporte', # Tipo de transporte relacionada ao fato
						'complemento', # Complemento do endereço
						'atuacao', # Atuação - Todos os valores constam como 'VITIMA'
						'meio_locomocao', # Meio de locomoção do autor
						'cor_veiculo', # Cor do veículo do autor
						'marca_veic_fuga', # Marca do veículo do autor
						'modelo_do_veic_fuga', # Modelo do veículo do autor
						'relatorio', # Relatório
						'qtd_autor', # Quantidade de autores relacionados à ocorrência
						'ident_autoria', # Se a identidade do autor é conhecida
						'consolidado', # Coluna que contém o valor 'HOMICIDIO' para todos os registros
						'fato_real', # Coluna que contém o valor 'HOMICIDIO' para todos os registros
						'meio_emp_deac', # Meio empregado, de acordo com a classificação da DEAC
						'causa_presumivel', # Causa presumível
						'especializacao_fato', # Especialização do fato
						'identificacao_fato', # Identificação do fato
						'vit_estado_civil', # Estado civil da vítima
						'aut_sexo', # Sexo do autor
						'aut_sit_emprego', # Situação empregatícia do autor
						'aut_est_civil' # Estado civil do autor
						]

raw_VCM = raw_VCM.drop(columns=colunas_para_remover)

In [ ]:
# Contagem do número de variáveis
v_raw_vcm = len(raw_VCM.columns)
# Contagem do número de registros
n_raw_vcm = len(raw_VCM)

print(f'O conjunto de dados possui {n_raw_vcm} registros e {v_raw_vcm} variáveis.')

In [ ]:
# Informações gerais do dataframe
raw_VCM.info()

In [ ]:
# Visualização do dataframe de VCM
raw_VCM.head(10)

#### Alteração dos tipos de variáveis

In [ ]:
print(raw_VCM['hora_registro'].str.slice(11, 19))

In [ ]:
# Converte as colunas necessárias para o formato de data
dados_VCM = raw_VCM
#dados_VCM.loc[:, 'data_registro'] = pd.to_datetime(dados_VCM['data_registro'])
#dados_VCM.loc[:, 'hora_registro'] = pd.to_datetime(raw_VCM['hora_registro'].astype('str').str.slice(11, 19), format='%H:%M:%S').dt.time
#dados_VCM.loc[:, 'data_fato'] = pd.to_datetime(dados_VCM['data_fato'])
#dados_VCM.loc[:, 'hora_fato'] = pd.to_datetime(raw_VCM['hora_fato'].astype('str').str.slice(11, 19), format='%H:%M:%S').dt.time
#dados_VCM.loc[:, 'vit_dt_nasc'] = pd.to_datetime(dados_VCM['vit_dt_nasc'])
#dados_VCM.loc[:, 'aut_data_nasc'] = pd.to_datetime(dados_VCM['aut_data_nasc'])

# Converte as colunas necessárias para o formato de data
dados_VCM = raw_VCM
dados_VCM['data_registro'] = pd.to_datetime(dados_VCM['data_registro'])
dados_VCM['hora_registro'] = pd.to_datetime(raw_VCM['hora_registro'].astype('str').str.slice(11, 19), format='%H:%M:%S').dt.time
dados_VCM['data_fato'] = pd.to_datetime(dados_VCM['data_fato'])
dados_VCM['hora_fato'] = pd.to_datetime(raw_VCM['hora_fato'].astype('str').str.slice(11, 19), format='%H:%M:%S').dt.time
dados_VCM['vit_dt_nasc'] = pd.to_datetime(dados_VCM['vit_dt_nasc'])
dados_VCM['aut_data_nasc'] = pd.to_datetime(dados_VCM['aut_data_nasc'])

In [ ]:
dados_VCM.head(10)

#### Adição de uma nova coluna referente à idade da vítima

In [ ]:
# Estatísticas descritivas da variáviel 'vit_idade'
dados_VCM['vit_idade'].describe()

In [ ]:
# Substituindo os valores anômalos da variável 'vit_idade' por nulo
dados_VCM.loc[dados_VCM['vit_idade'] > 105, 'vit_idade'] = None

In [ ]:
# Estatísticas descritivas da variáviel 'vit_idade' após a remoção dos valores anômalos
dados_VCM['vit_idade'].describe()

In [ ]:
# Se a data for menor ou igual a '1900-09-09', substitua por null, caso contrário, mantenha a data
dados_VCM.loc[dados_VCM['vit_dt_nasc'] <= '1900-09-09', 'vit_dt_nasc'] = None

In [ ]:
# Calcule a idade em anos com tratamento para valores nulos
dados_VCM['vit_idade_calculada'] = (dados_VCM['data_fato'] - dados_VCM['vit_dt_nasc']).dt.days // 365

In [ ]:
print(dados_VCM['vit_idade_calculada'].describe())
print(dados_VCM['vit_idade'].describe())

#### Idade do autor

In [ ]:
# Se a data for menor ou igual a '1900-09-09', substitua por null, caso contrário, mantenha a data
dados_VCM.loc[dados_VCM['aut_data_nasc'] <= '1900-09-09', 'aut_data_nasc'] = None

In [ ]:
# Calcule a idade em anos com tratamento para valores nulos
dados_VCM['aut_idade_calculada'] = (dados_VCM['data_fato'] - dados_VCM['aut_data_nasc']).dt.days // 365

In [ ]:
# Criando uma nova coluna referente à faixa etária
#dados_VCM['vit_idade_calculada'] = dados_VCM['vit_idade_calculada'].fillna('Desconhecida')

# Definindo os intervalos e os rótulos das faixas etárias
intervals = [0, 11, 17, 24, 29, 34, 64, 105]
labels = ['CRIANCA (0 A 11 ANOS)', 'ADOLESCENTE (12 A 17 ANOS)', 'ADULTO I (18 A 24 ANOS)', 'ADULTO II (25 A 29 ANOS)', 'ADULTO III (30 A 34 ANOS)', 'ADULTO IV (35 A 64 ANOS)', 'IDOSO']
categories = labels + ['Desconhecida']

dados_VCM['vit_faixa_etaria_calculada'] = pd.cut(dados_VCM['vit_idade_calculada'], bins=intervals, labels=labels, right=False)

In [ ]:
# Adiciona a categoria 'Desconhecida' à lista de categorias da faixa etária
dados_VCM['vit_faixa_etaria_calculada'] = pd.Categorical(dados_VCM['vit_faixa_etaria_calculada'], categories=categories)
# Preenche os registros com valores nulos como 'Desconhecida'
dados_VCM['vit_faixa_etaria_calculada'] = dados_VCM['vit_faixa_etaria_calculada'].fillna('Desconhecida')

In [ ]:
# Exibe a contagem das faixas etárias
dados_VCM['vit_faixa_etaria_calculada'].value_counts()

In [ ]:
dados_VCM['vit_faixa_etaria_calculada'].describe()

In [ ]:
dados_VCM['aut_faixa_etaria_calculada'] = pd.cut(dados_VCM['aut_idade_calculada'], bins=intervals, labels=labels, right=False)

# Adiciona a categoria 'Desconhecida' à lista de categorias da faixa etária
dados_VCM['aut_faixa_etaria_calculada'] = pd.Categorical(dados_VCM['aut_faixa_etaria_calculada'], categories=categories)
# Preenche os registros com valores nulos como 'Desconhecida'
dados_VCM['aut_faixa_etaria_calculada'] = dados_VCM['aut_faixa_etaria_calculada'].fillna('Desconhecida')

In [ ]:
# Exibe a contagem das faixas etárias
dados_VCM['aut_faixa_etaria_calculada'].value_counts()

In [ ]:
# Verifica a quantidade de casos nos quais as vítimas tinham mais de 18 anos
dados_VCM_maiores_18_PA = dados_VCM[dados_VCM['vit_idade_calculada'] >= 18]
len(dados_VCM_maiores_18_PA)

In [ ]:
# Verifica a quantidade total de registros
len(dados_VCM)

In [ ]:
# Cria um novo dataframe para ser salvo referente aos dados do estado do Pará
dados_VCM_PA = dados_VCM
# Visualização do dataframe
dados_VCM_PA.head(10)

#### Tratamento dos meses

In [ ]:
# Dicionário de mapeamento de meses para números
meses = {
    "JANEIRO": 1,
    "FEVEREIRO": 2,
    "MARCO": 3,
    "ABRIL": 4,
    "MAIO": 5,
    "JUNHO": 6,
    "JULHO": 7,
    "AGOSTO": 8,
    "SETEMBRO": 9,
    "OUTUBRO": 10,
    "NOVEMBRO": 11,
    "DEZEMBRO": 12
}

dados_VCM_PA['mes_registro'] = dados_VCM_PA['mes_registro'].replace(meses)
dados_VCM_PA['mes_fato'] = dados_VCM_PA['mes_fato'].replace(meses)

In [ ]:
dados_VCM_PA.head(10)

#### Faixa horária

In [ ]:
horas = dados_VCM_PA['hora_fato']

for h in horas:
	print(h.hour)
	break

In [ ]:

# Definindo os intervalos e os rótulos das faixas etárias
intervals = [0, 6, 12, 18, 24]
labels = ['Madrugada', 'Manhã', 'Tarde', 'Noite']
categories = labels + ['Outro']

# Cria uma nova coluna com a hora do fato no formato inteiro
dados_VCM_PA['hora_fato_int'] = dados_VCM_PA['hora_fato'].apply(lambda x: x.hour)

# Período que ocorreu o fato (Manhã, Tarde, Noite ou Madrugada)
dados_VCM_PA['faixa_horaria'] = pd.cut(dados_VCM_PA['hora_fato_int'], bins=intervals, labels=labels, right=False)
dados_VCM_PA['faixa_horaria'] = pd.Categorical(dados_VCM['faixa_horaria'], categories=categories)
dados_VCM_PA['faixa_horaria'] = dados_VCM['faixa_horaria'].fillna('Outro')

# Período que ocorreu o fato (['00h - 06h', '06h - 12h', '12h - 18h' ou '18h - 00h')
labels = ['00h - 06h', '06h - 12h', '12h - 18h', '18h - 00h']
categories = labels + ['Outro']

dados_VCM_PA['faixa_horaria_6h'] = pd.cut(dados_VCM_PA['hora_fato_int'], bins=intervals, labels=labels, right=False)
dados_VCM_PA['faixa_horaria_6h'] = pd.Categorical(dados_VCM['faixa_horaria_6h'], categories=categories)
dados_VCM_PA['faixa_horaria_6h'] = dados_VCM['faixa_horaria_6h'].fillna('Outro')

intervals = [0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24]
labels = ['00h - 02h', '02h - 04h', '04h - 06h', '06h - 08h', '08h - 10h', '10h - 12h', '12h - 14h', '14h - 16h', '16h - 18h', '18h - 20h', '20h - 22h', '22h - 00h']
categories = labels + ['Outro']

dados_VCM_PA['faixa_horaria_2h'] = pd.cut(dados_VCM_PA['hora_fato_int'], bins=intervals, labels=labels, right=False)
dados_VCM_PA['faixa_horaria_2h'] = pd.Categorical(dados_VCM['faixa_horaria_2h'], categories=categories)
dados_VCM_PA['faixa_horaria_2h'] = dados_VCM['faixa_horaria_2h'].fillna('Outro')

# Exibindo a coluna com o horário e as demais colunas criadas
dados_VCM_PA[['hora_fato', 'hora_fato_int', 'faixa_horaria', 'faixa_horaria_6h', 'faixa_horaria_2h']].head(10)

In [ ]:
# Remoção de mais colunas
colunas_para_remover = ['fx_4_hor',
					'fx_12_hr',
					'latitude',
					'longitude',
					'distrito',
					'regionais',
					'vit_idade',
					'vit_fx_etaria',
					'vit_sexo',
					'vit_situacao_emprego',
					'aut_idade',
					'aut_fx_etaria',
					'hora_fato_int'
				]

dados_VCM_PA = dados_VCM_PA.drop(columns=colunas_para_remover)

In [ ]:
# Contagem do número de variáveis
v_vcm_pa = len(dados_VCM_PA.columns)
# Contagem do número de registros
n_vcm_pa = len(dados_VCM_PA)

print(f'O conjunto de dados possui {n_vcm_pa} registros e {v_vcm_pa} variáveis.')

## Ordenação das colunas <a id="ordernacao-das-colunas"></a>

### Renomeando colunas

In [ ]:
dados_VCM_PA = dados_VCM_PA.rename(columns={
	'vit_idade_calculada'		: 'vit_idade',
	'aut_idade_calculada'		: 'aut_idade',
	'vit_faixa_etaria_calculada': 'vit_faixa_etaria',
	'aut_faixa_etaria_calculada': 'aut_faixa_etaria',
	'meio_empregado_sisp'		: 'meio_empregado',
	'municipios'				: 'municipio',
	'bairros'					: 'bairro',
	'aut_data_nasc'				: 'aut_dt_nasc'})

Cria uma nova coluna para especificar o estado do Pará.

In [ ]:
dados_VCM_PA['estado'] = 'PA'

In [ ]:
len(dados_VCM_PA.columns)

### Definindo a nova ordem das colunas

In [ ]:
dados_VCM_PA = dados_VCM_PA[['data_registro', 'hora_registro', 'data_fato', 'hora_fato', 'dia_semana', 'faixa_horaria_2h', 'faixa_horaria_6h', 'classe_motivo', 'mes_fato', 'ano_registro', 'ano_fato', 'registros', 'municipio', 'bairro', 'local_ocorrencia', 'meio_empregado', 'mes_registro', 'especificacao_crime', 'vit_dt_nasc', 'vit_idade', 'vit_faixa_etaria', 'vit_cor_pele', 'vit_grau_inst', 'vit_profissao', 'aut_dt_nasc', 'aut_idade', 'aut_faixa_etaria', 'aut_cor_pele', 'aut_grau_inst', 'aut_profissao', 'grau_de_relacionamento', 'faixa_horaria', 'estado']]

## Exportação dos dados formatados <a id="exportacao-dos-dados-formatados"></a>

In [ ]:
import datetime as dt # Manipulação de data

dados_VCM_PA.to_csv('./dados-tratados/' + dt.datetime.now().strftime('%Y%m%d%H%M%S') + '-dados_VCM_PA-pandas.csv', index=False, sep='|', encoding='utf-8')